In [16]:
import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString


import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

In [2]:
df = pd.read_json("../scripts/Metropolis_CVE.json")

In [3]:
mpos_folder = 'mpos_2020'

In [4]:
for c in df.columns.unique():
        if c == 'Guadalajara':
            mun_gdf = gpd.GeoDataFrame()
            for i in range(len(df.loc["mpos", c])):
                # Extracts specific municipality code
                m = df.loc["mpos", c][i]
                # Downloads municipality polygon according to code
                query = f"SELECT * FROM marco.{mpos_folder} WHERE \"CVEGEO\" LIKE \'{m}\'"
                mun_gdf = mun_gdf.append(aup.gdf_from_query(query, geometry_col='geometry'))

In [5]:
mun_gdf = mun_gdf.set_crs("EPSG:4326")

In [6]:
G, nodes, edges = aup.graph_from_hippo(mun_gdf, 'osmnx')

In [7]:
grl_path = '../data/external/MDE/'
mde_path = [] # list to append mde path strings
c='Guadalajara'
#Gathers state codes for MDE
for i in range(len(df.loc["edo", c])):
    e = df.loc["edo", c][i]
    tmp_path = grl_path + (f'CEM_V3_20170619_R15_E{e}_TIF/')

    #search for files in tmp_path for .tif
    for f in os.listdir(tmp_path):
        if f.endswith('.tif'):
            mde_path.append(tmp_path+f)

In [10]:
mde_path

['../data/external/MDE/CEM_V3_20170619_R15_E14_TIF/Jalisco_r15m.tif']

In [31]:
nodes_tmp = nodes.reset_index().copy()
nodes_tmp

,osmid,x,y,street_count,geometry
0,28751344,-103.306309,20.711533,3,POINT (-103.30631 20.71153)
1,28753224,-103.316645,20.700650,3,POINT (-103.31665 20.70065)
2,28757751,-103.307356,20.708526,3,POINT (-103.30736 20.70853)
3,28757777,-103.306806,20.710093,4,POINT (-103.30681 20.71009)
4,28768905,-103.305728,20.696546,3,POINT (-103.30573 20.69655)
...,...,...,...,...,...
173954,8949749314,-103.383125,20.538486,3,POINT (-103.38312 20.53849)
173955,8949749315,-103.383062,20.538581,3,POINT (-103.38306 20.53858)
173956,8949749363,-103.430458,20.544440,3,POINT (-103.43046 20.54444)
173957,8949749364,-103.430610,20.544488,3,POINT (-103.43061 20.54449)


In [63]:
nodes_dict = nodes_tmp.to_dict()
df_final = pd.DataFrame.from_dict(nodes_dict)
df_final

,osmid,x,y,street_count,geometry
0,28751344,-103.306309,20.711533,3,POINT (-103.3063086 20.7115326)
1,28753224,-103.316645,20.700650,3,POINT (-103.3166454 20.7006495)
2,28757751,-103.307356,20.708526,3,POINT (-103.3073563 20.7085262)
3,28757777,-103.306806,20.710093,4,POINT (-103.3068063 20.7100932)
4,28768905,-103.305728,20.696546,3,POINT (-103.3057279 20.6965463)
...,...,...,...,...,...
173954,8949749314,-103.383125,20.538486,3,POINT (-103.3831249 20.5384863)
173955,8949749315,-103.383062,20.538581,3,POINT (-103.3830624 20.538581)
173956,8949749363,-103.430458,20.544440,3,POINT (-103.4304584 20.5444397)
173957,8949749364,-103.430610,20.544488,3,POINT (-103.4306101 20.544488)


In [69]:
nodes_dict['osmid'][len(nodes_dict['osmid'])] = 10000

In [70]:
nodes_dict['osmid'][len(nodes_dict['osmid'])-1]

10000

In [35]:
nodes_tmp = nodes.reset_index().copy()

edges_tmp = edges.reset_index().copy()

from_osmid = list(set(edges_tmp['u'].to_list()).difference(set(nodes_tmp.osmid.to_list())))

for i in from_osmid:
    row = edges_tmp.loc[(edges_tmp.u==i)].iloc[0]
    coords = [(coords) for coords in list(row['geometry'].coords)]
    first_coord, last_coord = [ coords[i] for i in (0, -1) ]
    
    missing_osmid = {'osmid':i,'x':first_coord[0],
                          'y':first_coord[1], 'street_count':np.nan,
                          'geometry':Point(first_coord)}
    
to_osmid = list(set(edges_tmp['v'].to_list()).difference(set(nodes_tmp.osmid.to_list())))

for i in to_osmid:
    row = edges_tmp.loc[(edges_tmp.u==i)].iloc[0]
    coords = [(coords) for coords in list(row['geometry'].coords)]
    first_coord, last_coord = [ coords[i] for i in (0, -1) ]
    
    nodes_tmp.iloc[-1] = [i,last_coord[0],
                          last_coord[1], np.nan,
                          Point(last_coord)]

<__array_function__ internals>:5: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
<__array_function__ internals>:5: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengt

In [57]:
df_missing = pd.DataFrame([[i,first_coord[0],first_coord[1],np.nan,Point(first_coord)]],
                             columns=['osmid','x','y','street_count','geometry'])
nodes_tmp.concat(df_missing)

AttributeError: 'GeoDataFrame' object has no attribute 'concat'

In [ ]:
nodes_tmp = nodes.reset_index().copy()

edges_tmp = edges.reset_index().copy()

from_osmid = list(set(edges_tmp['u'].to_list()).difference(set(nodes_tmp.osmid.to_list())))


for i in from_osmid:
    row = edges_tmp.loc[(edges_tmp.u==i)].iloc[0]
    coords = [(coords) for coords in list(row['geometry'].coords)]
    first_coord, last_coord = [ coords[i] for i in (0, -1) ]
    
    missing_osmid = {'osmid':i,'x':first_coord[0],
                          'y':first_coord[1], 'street_count':np.nan,
                          'geometry':Point(first_coord)}
    df_missing = pd.DataFrame([i,first_coord[0],first_coord[1],np.nan,Point(first_coord)],
                             columns=['osmid','x','y','street_count','geometry'])
    
    nodes_dict.append(missing_osmid)
    
list(nodes_dict['osmid'].values())
    
to_osmid = list(set(edges_tmp['v'].to_list()).difference(set(nodes_tmp.osmid.to_list())))

for i in to_osmid:
    row = edges_tmp.loc[(edges_tmp.u==i)].iloc[0]
    coords = [(coords) for coords in list(row['geometry'].coords)]
    first_coord, last_coord = [ coords[i] for i in (0, -1) ]
    
    missing_osmid = {'osmid':i,'x':last_coord[0],
                          'y':last_coord[1], 'street_count':np.nan,
                          'geometry':Point(last_coord)}
    
    nodes_dict.append(missing_osmid)
    
nodes_tmp = pd.DataFrame.from_dict(nodes_dict)

In [78]:
nodes_tmp = nodes.reset_index().copy()

edges_tmp = edges.reset_index().copy()

from_osmid = list(set(edges_tmp['u'].to_list()).difference(set(nodes_tmp.osmid.to_list())))

nodes_dict = nodes_tmp.to_dict()

for i in from_osmid:
    row = edges_tmp.loc[(edges_tmp.u==i)].iloc[0]
    coords = [(coords) for coords in list(row['geometry'].coords)]
    first_coord, last_coord = [ coords[i] for i in (0, -1) ]
    
    nodes_dict['osmid'][len(nodes_dict['osmid'])] = i
    nodes_dict['x'][len(nodes_dict['x'])] = first_coord[0]
    nodes_dict['y'][len(nodes_dict['y'])] = first_coord[1]
    nodes_dict['street_count'][len(nodes_dict['street_count'])] = np.nan
    nodes_dict['geometry'][len(nodes_dict['geometry'])] = Point(first_coord)
        
    
to_osmid = list(set(edges_tmp['v'].to_list()).difference(set(list(nodes_dict['osmid'].values()))))

for i in to_osmid:
    row = edges_tmp.loc[(edges_tmp.u==i)].iloc[0]
    coords = [(coords) for coords in list(row['geometry'].coords)]
    first_coord, last_coord = [ coords[i] for i in (0, -1) ]
    
    nodes_dict['osmid'][len(nodes_dict['osmid'])] = i
    nodes_dict['x'][len(nodes_dict['x'])] = last_coord[0]
    nodes_dict['y'][len(nodes_dict['y'])] = last_coord[1]
    nodes_dict['street_count'][len(nodes_dict['street_count'])] = np.nan
    nodes_dict['geometry'][len(nodes_dict['geometry'])] = Point(last_coord)
    
nodes_tmp = pd.DataFrame.from_dict(nodes_dict)
nodes_tmp = gpd.GeoDataFrame(nodes_tmp, crs="EPSG:4326", geometry='geometry')

In [79]:
nodes_tmp

,osmid,x,y,street_count,geometry
0,28751344,-103.306309,20.711533,3.0,POINT (-103.30631 20.71153)
1,28753224,-103.316645,20.700650,3.0,POINT (-103.31665 20.70065)
2,28757751,-103.307356,20.708526,3.0,POINT (-103.30736 20.70853)
3,28757777,-103.306806,20.710093,4.0,POINT (-103.30681 20.71009)
4,28768905,-103.305728,20.696546,3.0,POINT (-103.30573 20.69655)
...,...,...,...,...,...
173959,8404816004,-103.408646,20.544511,NaN,POINT (-103.40865 20.54451)
173960,8389432427,-103.473608,20.539874,NaN,POINT (-103.47361 20.53987)
173961,1776712044,-103.420512,20.546057,NaN,POINT (-103.42051 20.54606)
173962,6289765594,-103.406357,20.543847,NaN,POINT (-103.40636 20.54385)


In [80]:
G_new = ox.graph_from_gdfs(nodes_tmp.set_index('osmid'), edges_tmp.set_index(['u','v','key']))

In [81]:
G_elev_mde = ox.elevation.add_node_elevations_raster(G_new, mde_path)

In [82]:
G_elev_mde = ox.elevation.add_edge_grades(G_elev_mde, add_absolute=True, precision=3)

In [83]:
nodes_elev_mde, edges_elev_mde = ox.graph_to_gdfs(G_elev_mde, nodes=True, edges=True)

In [84]:
print(nodes_elev_mde.shape)
nodes_elev_mde.head(4)

(173964, 5)


,x,y,street_count,elevation,geometry
osmid,,,,,
1754406771,-103.402106,20.500385,3.0,1550.0,POINT (-103.40211 20.50038)
1754406780,-103.400626,20.500664,3.0,1547.0,POINT (-103.40063 20.50066)
1754406772,-103.397833,20.500383,4.0,1539.0,POINT (-103.39783 20.50038)
1754406789,-103.398871,20.500910,3.0,1540.0,POINT (-103.39887 20.50091)


In [85]:
print(edges_elev_mde.shape)
edges_elev_mde.head(4)

(419221, 17)


osmid  oneway                    name  \
u          v          key                                              
1754406771 1754406780 0    163760869   False  Calle Cumbre del Marco   
           1754406763 0    163760852   False    Calle Cumbre Córdoba   
           6459500675 0    163760852   False    Calle Cumbre Córdoba   
1754406780 1754406771 0    163760869   False  Calle Cumbre del Marco   

                               highway   length  \
u          v          key                         
1754406771 1754406780 0    residential  157.211   
           1754406763 0    residential   41.093   
           6459500675 0    residential    4.747   
1754406780 1754406771 0    residential  157.211   

                                                                    geometry  \
u          v          key                                                      
1754406771 1754406780 0    LINESTRING (-103.40211 20.50038, -103.40063 20...   
           1754406763 0    LINESTRING (-103.40211 20.50038, -103.40220 20...   
           6459500675 0    LINESTRING (-103.40211 20.50038, -103.40209 20...   
1754406780 1754406771 0    LINESTRING (-103.40063 20.50066, -103.40211 20...   

                           grade  grade_abs junction maxspeed  ref bridge  \
u          v          key                                                   
1754406771 1754406780 0   -0.019      0.019      NaN      NaN  NaN    NaN   
           1754406763 0   -0.024      0.024      NaN      NaN  NaN    NaN   
           6459500675 0    0.000      0.000      NaN      NaN  NaN    NaN   
1754406780 1754406771 0    0.019      0.019      NaN      NaN  NaN    NaN   

                          lanes access tunnel width service  
u          v          key                                    
1754406771 1754406780 0     NaN    NaN    NaN   NaN     NaN  
           1754406763 0     NaN    NaN    NaN   NaN     NaN  
           6459500675 0     NaN    NaN    NaN   NaN     NaN  
1754406780 1754406771 0     NaN    NaN    NaN   NaN     NaN

In [54]:
nodes_dict

[{'osmid': {0: 28751344,
   1: 28753224,
   2: 28757751,
   3: 28757777,
   4: 28768905,
   5: 28769466,
   6: 28771230,
   7: 28772094,
   8: 28772132,
   9: 28774831,
   10: 28774889,
   11: 28775949,
   12: 28776032,
   13: 28776223,
   14: 28776246,
   15: 28776279,
   16: 28777191,
   17: 28777954,
   18: 28781279,
   19: 28781298,
   20: 28781333,
   21: 28781359,
   22: 28781433,
   23: 28781516,
   24: 28781517,
   25: 28781518,
   26: 28781520,
   27: 28781522,
   28: 28781523,
   29: 28781524,
   30: 28781529,
   31: 28781530,
   32: 28791285,
   33: 28791315,
   34: 28791320,
   35: 28791354,
   36: 28791359,
   37: 28791397,
   38: 28791518,
   39: 28791819,
   40: 130153606,
   41: 130153633,
   42: 130153676,
   43: 130153753,
   44: 130153815,
   45: 130154244,
   46: 251530643,
   47: 251530655,
   48: 251538467,
   49: 251538474,
   50: 267537966,
   51: 267537978,
   52: 267537985,
   53: 267537992,
   54: 267538751,
   55: 267538775,
   56: 267538810,
   57: 26753881

In [37]:
from_osmid

[8404816004, 8389432427, 1776712044, 6289765594, 8778125631]

In [38]:
nodes_tmp

,osmid,x,y,street_count,geometry
0,28751344,-103.306309,20.711533,3.0,POINT (-103.30631 20.71153)
1,28753224,-103.316645,20.700650,3.0,POINT (-103.31665 20.70065)
2,28757751,-103.307356,20.708526,3.0,POINT (-103.30736 20.70853)
3,28757777,-103.306806,20.710093,4.0,POINT (-103.30681 20.71009)
4,28768905,-103.305728,20.696546,3.0,POINT (-103.30573 20.69655)
...,...,...,...,...,...
173954,8949749314,-103.383125,20.538486,3.0,POINT (-103.38312 20.53849)
173955,8949749315,-103.383062,20.538581,3.0,POINT (-103.38306 20.53858)
173956,8949749363,-103.430458,20.544440,3.0,POINT (-103.43046 20.54444)
173957,8949749364,-103.430610,20.544488,3.0,POINT (-103.43061 20.54449)


In [ ]:
mde_path = ['../data/external/MDE/tmp/MDE_AMG.tif']

In [ ]:
G_elev_mde = ox.elevation.add_node_elevations_raster(G, mde_path)

In [49]:
#Creates wkt for query
gdf_tmp = mun_gdf.copy()
gdf_tmp = gdf_tmp.to_crs("EPSG:6372")
gdf_tmp = gdf_tmp.buffer(1000).reset_index().rename(columns={0:'geometry'})
gdf_tmp = gdf_tmp.to_crs("EPSG:4326")
gdf_tmp = gdf_tmp.dissolve()

In [50]:
poly = gdf_tmp.geometry
# Extracts coordinates from polygon as DataFrame
coord_val = poly.bounds
# Gets coordinates for bounding box
n = coord_val.maxy.max()
s = coord_val.miny.min()
e = coord_val.maxx.max()
w = coord_val.minx.min()

# Downloads OSMnx graph from bounding box
G_ox = ox.graph_from_bbox(n, s, e, w, network_type="all")

/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/osmnx/utils_geo.py:335: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/osmnx/utils_geo.py:426: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:
/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/osmnx/utils_geo.py:426: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


In [57]:
nodes_ox, edges_ox = ox.graph_to_gdfs(G_ox)
print(nodes_ox.shape)
nodes_ox.head(3)

(188399, 5)


,y,x,highway,street_count,geometry
osmid,,,,,
28781524,20.672484,-103.330522,traffic_signals,4,POINT (-103.33052 20.67248)
130153606,20.669240,-103.341135,NaN,4,POINT (-103.34114 20.66924)
267537978,20.628507,-103.291851,NaN,3,POINT (-103.29185 20.62851)


In [52]:
print(nodes.shape)
nodes.head(2)

(173959, 4)


,x,y,street_count,geometry
osmid,,,,
28751344,-103.306309,20.711533,3,POINT (-103.30631 20.71153)
28753224,-103.316645,20.700650,3,POINT (-103.31665 20.70065)


In [58]:
nodes_ox['cont'] = 1

In [59]:
nodes_merge = nodes.merge(nodes_ox, left_index=True, right_index=True, how='left')
print(nodes_merge.shape)
nodes_merge.head(2)

(173959, 10)


,x_x,y_x,street_count_x,geometry_x,y_y,x_y,highway,street_count_y,geometry_y,cont
osmid,,,,,,,,,,
28751344,-103.306309,20.711533,3,POINT (-103.30631 20.71153),20.711533,-103.306309,NaN,3.0,POINT (-103.30631 20.71153),1.0
28753224,-103.316645,20.700650,3,POINT (-103.31665 20.70065),20.700650,-103.316645,NaN,3.0,POINT (-103.31665 20.70065),1.0


In [60]:
nodes_merge['cont'].sum()

173838.0

In [ ]:
173836

In [36]:
nodes_ox['cont'].sum()

183664

In [64]:
nodes_merge.loc[nodes_merge['cont'] != 1]

,x_x,y_x,street_count_x,geometry_x,y_y,x_y,highway,street_count_y,geometry_y,cont
osmid,,,,,,,,,,
273160154,-103.323784,20.720513,3,POINT (-103.32378 20.72051),NaN,NaN,NaN,NaN,None,NaN
1339256735,-103.351231,20.666336,3,POINT (-103.35123 20.66634),NaN,NaN,NaN,NaN,None,NaN
1341930150,-103.350442,20.666203,4,POINT (-103.35044 20.66620),NaN,NaN,NaN,NaN,None,NaN
1387702265,-103.415865,20.669991,2,POINT (-103.41587 20.66999),NaN,NaN,NaN,NaN,None,NaN
1478445184,-103.328907,20.722321,3,POINT (-103.32891 20.72232),NaN,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...
8776319151,-103.492940,20.488133,1,POINT (-103.49294 20.48813),NaN,NaN,NaN,NaN,None,NaN
8778233574,-103.479082,20.526219,3,POINT (-103.47908 20.52622),NaN,NaN,NaN,NaN,None,NaN
8778274053,-103.481995,20.519208,3,POINT (-103.48199 20.51921),NaN,NaN,NaN,NaN,None,NaN


In [67]:
nodes_ox_tmp = nodes_ox.reset_index()
nodes_ox_tmp.loc[nodes_ox_tmp.osmid==1478445184]

,osmid,y,x,highway,street_count,geometry,cont


In [72]:
edges_ox_tmp = edges_ox.reset_index()
edges_ox_tmp.loc[edges_ox_tmp.v == 1478445184]

,u,v,key,osmid,oneway,lanes,name,highway,maxspeed,length,ref,bridge,geometry,tunnel,junction,access,service,width


In [77]:
edges_tmp = edges.reset_index()
edges_tmp.loc[edges_tmp.v == 1478445184]

,u,v,key,osmid,oneway,lanes,name,highway,maxspeed,length,geometry,bridge,ref,junction,tunnel,access,width,service
51344,6133577229,1478445184,0,375240374,True,4,Anillo Periférico Norte Manuel Gómez Morín,trunk,80,87.488,"LINESTRING (-103.32968 20.72264, -103.32891 20...",None,None,None,None,None,None,None
51494,1728433966,1478445184,0,134477484,False,None,Calle José María Chávez,residential,None,61.308,"LINESTRING (-103.32892 20.72177, -103.32891 20...",None,None,None,None,None,None,None


In [87]:
edges_tmp.loc[(edges_tmp.u==6133577229)&(edges_tmp.v==1478445184)]

,u,v,key,osmid,oneway,lanes,name,highway,maxspeed,length,geometry,bridge,ref,junction,tunnel,access,width,service
51344,6133577229,1478445184,0,375240374,True,4,Anillo Periférico Norte Manuel Gómez Morín,trunk,80,87.488,"LINESTRING (-103.32968 20.72264, -103.32891 20...",None,None,None,None,None,None,None


In [135]:
nodes_tmp = nodes.reset_index().copy()
edges_tmp.loc[edges_tmp.u is in nodes_tmp.osmid.to_list()]

SyntaxError: invalid syntax (<ipython-input-135-7ea7a973e08f>, line 2)

In [136]:
nodes_tmp = nodes.reset_index().copy()

[edges_tmp['u'].to_list() not in nodes_tmp.osmid.to_list()]

[True]

In [101]:
res = [x for x in edges_tmp['u'].to_list() if x not in nodes_tmp.osmid.to_list()]

KeyboardInterrupt: 

In [105]:
set(edges_tmp['u'].to_list()).difference(set(nodes_tmp.osmid.to_list()))

{1776712044, 6289765594, 8389432427, 8404816004, 8778125631}

In [106]:
set(edges_tmp['v'].to_list()).difference(set(nodes_tmp.osmid.to_list()))

{1776712044, 6289765594, 8389432427, 8404816004, 8778125631}

In [118]:
from shapely.geometry import Point, LineString

row = edges_tmp.loc[(edges_tmp.u==1776712044)].iloc[0]
coords = [(coords) for coords in list(row['geometry'].coords)]
first_coord, last_coord = [ coords[i] for i in (0, -1) ]

In [119]:
first_coord

(-103.4205117, 20.5460571)

In [120]:
last_coord

(-103.4207007, 20.5460901)

In [121]:
row = edges_tmp.loc[(edges_tmp.v==1776712044)].iloc[0]
coords = [(coords) for coords in list(row['geometry'].coords)]
first_coord, last_coord = [ coords[i] for i in (0, -1) ]
last_coord

(-103.4205117, 20.5460571)

In [122]:
nodes_tmp

,osmid,x,y,street_count,geometry
0,28751344,-103.306309,20.711533,3,POINT (-103.30631 20.71153)
1,28753224,-103.316645,20.700650,3,POINT (-103.31665 20.70065)
2,28757751,-103.307356,20.708526,3,POINT (-103.30736 20.70853)
3,28757777,-103.306806,20.710093,4,POINT (-103.30681 20.71009)
4,28768905,-103.305728,20.696546,3,POINT (-103.30573 20.69655)
...,...,...,...,...,...
173954,8949749314,-103.383125,20.538486,3,POINT (-103.38312 20.53849)
173955,8949749315,-103.383062,20.538581,3,POINT (-103.38306 20.53858)
173956,8949749363,-103.430458,20.544440,3,POINT (-103.43046 20.54444)
173957,8949749364,-103.430610,20.544488,3,POINT (-103.43061 20.54449)


In [141]:
nodes_tmp = nodes.reset_index().copy()

nodes_tmp.loc[len(nodes_tmp)] = [1776712044, last_coord[0], last_coord[1], np.nan, Point(last_coord)]

/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [142]:
nodes_tmp

,osmid,x,y,street_count,geometry
0,28751344,-103.306309,20.711533,3.0,POINT (-103.30631 20.71153)
1,28753224,-103.316645,20.700650,3.0,POINT (-103.31665 20.70065)
2,28757751,-103.307356,20.708526,3.0,POINT (-103.30736 20.70853)
3,28757777,-103.306806,20.710093,4.0,POINT (-103.30681 20.71009)
4,28768905,-103.305728,20.696546,3.0,POINT (-103.30573 20.69655)
...,...,...,...,...,...
173955,8949749315,-103.383062,20.538581,3.0,POINT (-103.38306 20.53858)
173956,8949749363,-103.430458,20.544440,3.0,POINT (-103.43046 20.54444)
173957,8949749364,-103.430610,20.544488,3.0,POINT (-103.43061 20.54449)
173958,8957012002,-103.406694,20.543913,3.0,POINT (-103.40669 20.54391)


In [164]:
nodes_tmp = nodes.reset_index().copy()

from_osmid = list(set(edges_tmp['u'].to_list()).difference(set(nodes_tmp.osmid.to_list())))
from_osmid

[8404816004, 8389432427, 1776712044, 6289765594, 8778125631]

In [165]:
edges_tmp = edges.reset_index().copy()

for i in from_osmid:
    row = edges_tmp.loc[(edges_tmp.u==i)].iloc[0]
    coords = [(coords) for coords in list(row['geometry'].coords)]
    first_coord, last_coord = [ coords[i] for i in (0, -1) ]
    nodes_tmp.loc[len(nodes_tmp)] = [i, first_coord[0], first_coord[1], np.nan, Point(first_coord)]

/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:118: Shape

In [180]:
df_tmp

,osmid,x,y,street_count,geometry
0,8778125631,-103.403861,20.542851,NaN,POINT (-103.4038613 20.5428512)


In [181]:
nodes_tmp

,osmid,x,y,street_count,geometry
0,28751344,-103.306309,20.711533,3.0,POINT (-103.30631 20.71153)
1,28753224,-103.316645,20.700650,3.0,POINT (-103.31665 20.70065)
2,28757751,-103.307356,20.708526,3.0,POINT (-103.30736 20.70853)
3,28757777,-103.306806,20.710093,4.0,POINT (-103.30681 20.71009)
4,28768905,-103.305728,20.696546,3.0,POINT (-103.30573 20.69655)
...,...,...,...,...,...
0,8404816004,-103.408646,20.544511,NaN,POINT (-103.40865 20.54451)
0,8389432427,-103.473608,20.539874,NaN,POINT (-103.47361 20.53987)
0,1776712044,-103.420512,20.546057,NaN,POINT (-103.42051 20.54606)
0,6289765594,-103.406357,20.543847,NaN,POINT (-103.40636 20.54385)


In [152]:
new_row = {'osmid':i, 'x':first_coord[0], 'y':first_coord[1], 'street_count':np.nan,'geometry':Point(first_coord)}
#append row to the dataframe
nodes_tmp = nodes_tmp.append(new_row, ignore_index=True)
nodes_tmp

/home/edgar/anaconda3/envs/geo_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


,osmid,x,y,street_count,geometry
0,28751344,-103.306309,20.711533,3.0,POINT (-103.30631 20.71153)
1,28753224,-103.316645,20.700650,3.0,POINT (-103.31665 20.70065)
2,28757751,-103.307356,20.708526,3.0,POINT (-103.30736 20.70853)
3,28757777,-103.306806,20.710093,4.0,POINT (-103.30681 20.71009)
4,28768905,-103.305728,20.696546,3.0,POINT (-103.30573 20.69655)
...,...,...,...,...,...
173962,6289765594,-103.406357,20.543847,NaN,POINT (-103.40636 20.54385)
173963,8778125631,-103.403861,20.542851,NaN,POINT (-103.40386 20.54285)
173964,8778125631,-103.403861,20.542851,NaN,POINT (-103.40386 20.54285)
173965,8778125631,-103.403861,20.542851,NaN,POINT (-103.40386 20.54285)


In [167]:
to_osmid = list(set(edges_tmp['v'].to_list()).difference(set(nodes_tmp.osmid.to_list())))
to_osmid

[]

In [168]:
G_new = ox.graph_from_gdfs(nodes_tmp.set_index('osmid'), edges_tmp.set_index(['u','v','key']))

In [169]:
mde_path = ['../data/external/MDE/tmp/MDE_AMG.tif']
G_elev_mde = ox.elevation.add_node_elevations_raster(G_new, mde_path)

In [159]:
len(G)

173964

In [171]:
nodes_elev = ox.graph_to_gdfs(G_elev_mde, edges=False, node_geometry=False)
print(nodes_elev.shape)
nodes_elev.head(5)

(173964, 4)


,x,y,street_count,elevation
osmid,,,,
1754406771,-103.402106,20.500385,3.0,1550.0
1754406780,-103.400626,20.500664,3.0,1548.0
1754406772,-103.397833,20.500383,4.0,1539.0
1754406789,-103.398871,20.500910,3.0,1540.0
1754406783,-103.397655,20.500702,4.0,1540.0
